In [2]:
from typing import Dict
import polars
import jsonlines
import pandas as pd
from pandas.api.types import is_numeric_dtype

import spacy
from spacy.lang.en import English

from code.utils.settings import DefaultPath
from code.utils.utils import rebuild_table

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/giovanni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nlp = spacy.load('en_core_web_sm')

NUMERICAL_NER_TAGS = {'DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'CARDINAL'}

def infer_column_type_from_sampled_value(sample_value_entry: Dict) -> str:
    if not 'ner_tags' in sample_value_entry:
        return 'text'

    num_numerical_tags = 0
    num_other_tags = 0

    for tag in sample_value_entry['ner_tags']:
        if tag in NUMERICAL_NER_TAGS:
            num_numerical_tags += 1
        else:
            num_other_tags += 1

    return 'real' if num_numerical_tags >= num_other_tags else 'text'


In [4]:
TABLES_DIR = DefaultPath.data_path.wikitables + 'threshold_r5-c2-a50/'
TABLES_FILE = TABLES_DIR + 'sloth-tables-r5-c2-a50.jsonl'

ntables_to_load_as_set = 10

json_tables = []
with jsonlines.open(TABLES_FILE) as table_reader:
    for i, json_table in enumerate(table_reader):
        if i >= ntables_to_load_as_set:
            break
        json_tables.append(json_table)

In [5]:
t = rebuild_table(json_tables[0])
t

,Pos,Driver,MNZ,NUR,ADR,VAL,Pts
0,1,Giuliano Alessi,3,2,1,1,75
1,2,Alessandro Balzan,Ret,1,3,3,49
2,3,Alessandro Battaglin,2,,2,4,41
3,4,Kristian Ghedina,5,8,6,2,39
4,5,Max Pigoli,1,3,Ret,10,35
5,6,Marco Gregori,4,,,5,18
6,7,Francesco Ascani,6,5,Ret,7,18
7,8,Mauro Simoncini,,7,4,,14
8,9,Moreno Petrini,Ret,4,Ret,,10
9,10,Corrado Canneori,,,5,,8


In [55]:
t['World Cup Finals']

,World Cup Finals,World Cup Finals,World Cup Finals,World Cup Finals,World Cup Finals,World Cup Finals,World Cup Finals,World Cup Finals,World Cup Finals
0,1930 to 1970,Did not enter,-,-,-,-,-,-,-
1,1974,Did not qualify,-,-,-,-,-,-,-
2,1978,Round 1,14th,3,0,1,2,8,Squad
3,1982,Withdrew,-,-,-,-,-,-,-
4,1986,Disqualified,-,-,-,-,-,-,-
5,1990,Did not qualify,-,-,-,-,-,-,-
6,1994,Did not qualify,-,-,-,-,-,-,-
7,1998,Round 1,20th,3,1,0,2,4,Squad
8,| 2002,Did not qualify,-,-,-,-,-,-,-
9,2006,Round 1,25th,3,0,1,2,6,Squad


In [51]:
for c in t.columns:
    print(c)
    t[c].to_list()

Club performance


AttributeError: 'DataFrame' object has no attribute 'to_list'

In [46]:
t = t.convert_dtypes()

In [48]:
t[1]

KeyError: 1

In [37]:
list(nlp.pipe(t['Pos'].to_list()))[0][0].ent_type_

'CARDINAL'